<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95_7%EC%9E%A5_%EC%9D%BC%EB%B0%98%ED%99%94%EC%84%A0%ED%98%95%EB%AA%A8%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로지스틱회귀모형
* <표 7.1> Sugar Glider(날다람쥐) 데이터
* occurr(Y) : 출현여부(1 or 0) 
* p_size_km(X1) : 구획의 크기
* con_metric(X2) : 다른 구획과의 연결성 정도

In [ ]:
#데이터 가져오기
glider_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/sugar_glider_binomial.csv'
glider = read.csv(glider_url, header = T)
head(glider, 3)

> 반응변수의 확률분포의 연결함수를 지정하는 명령어
* 정규분포 - 항등함수 : gaussian(link = identity)
* 이항분포 - 로짓함수 : binomial(link = logit)
* 포아송분포 - 로그함수 : poisson(link = log)
* 감마분포 - 역수함수 : Gamma(link = inverse)
* 준이항분포 - 로짓함수 : quasibinomial(link = logit)
* 준포아송분포 - 로그함수 : quasipoisson(link = log)

In [ ]:
#회귀모형 적합
glider.logit_m1 = glm(occurr ~ p_size_km + con_metric, family = binomial(link=logit), data = glider)
summary(glider.logit_m1)

> summary
1. call : 분석모형
2.  Deviance Residuals : 이탈도잔차의 최솟값, 사분위수, 최대값 등
* 이때 가정한 분석모형이 자료에 적합하다면 이탈도잔차는  근사적으로 정규분포를 따름
3.  Coefficients : 회귀계수 추정치와 검정결과
* 적합도된 회귀식 : log(^π / 1-^π ) = -3.606 + 0.024 X1 + 1.632 X2
* X1의 유의확률 : 약 0.002(0.00159) -> 매우 유의함
* H0 : log(π/1-π) = B0 + B2x2(즉,b1 =0) : Wald검정을 수행한 결과를 의미함
* 즉. 다른 구획과의 연결성정도(X2)를 보정한 후 구획의 크기(X1)가 Sugar Glider 출현 여부에 영향을 주는지 검정한 것 : X2는 X1이 보정된 후에는 유의하지 않음을 알 수 있다
4. dispersion parameter : 산포모수(이항분포에서 산포모수는 1임)
5. deviance & AIC(Akaike Information Criterion) : 이탈도 & 아카이케 정보기준
* 일반화선형모형에서 이탈도는 선형회귀모형의 잔차제곱합에 대응하는 값
* 즉. 주어진 자료와 적합된 모형 간의 편차를 정량한 값
* 비교하는 두 모형이 지분관계가 있으면 두 모형의 이탈도 차이는 귀무가설이 사실일 때 근사적으로 카이제곱분포를 따른다고 알려져 있음
* 이때 카이제곱분포의 자유도는 두 모형에서 추정되는 모수 개수의 차이임
* Numm deviance : 설명변수를 포함하지 않는 모형, log(ㅠ/1-ㅠ) = B0(H0 : 둘 다 0)에서 이탈값
* Residual deviance : 현재의 분석모형, log(ㅠ/1-ㅠ) = B0 + B1x1 + B2x2(H1 : 적어도 하나는 0이 아님)
* AIC값이 작을 수록 모형적합도가 높다(최적모형을 선택할 때 모형적합도를 상호비교할 때 사용됨)

In [ ]:
#모형의 유의성의 유의확률 : pchisq(LR, 자유도)
1-pchisq(68.994 - 54.661, 2)

* 이탈도에 근거한 검정통계량 LR = [Residual deviance] - [Null deviance]
* 관측값이 lr일 때 유의확률 p값 = Pr(LR >= lr)
* 이때 LR은 자유도 2인 카이제곱분포를 따르는 확률변수임
* 유의확률이 0.0008로 대립가설 채택

In [ ]:
#두 검정결과 비교
glider.logit_m0 = glm(occurr ~ 1, family = binomial(link = logit), data = glider)
anova(glider.logit_m0, glider.logit_m1, test = 'Chisq')

* 두 검정결과가 같음을 알 수 있음

* 반응변수 Y가 이항분포를 따르고 현재의 모형이 적합할 때, 자료의 수가 충분히 크다면 이탈도는 근사적으로 X2분포를 따른다고 알려짐
* 따라서 Residual deviance가 작다(자유도와 비슷한 값일 경우) -> 현재의 모형이 자료에 적합하다

> 이탈도 : 54.661, 자유도 47 -> 적합결여검정
* H0 : 현재모형
* H1 : 포화모형

In [ ]:
#이탈도에 근거한 모형의 적합결여검정
1-pchisq(54.661, 47)

* 0.206이므로 유의수준 0.05보다 크므로 현재모형의 적합결여는 통계적으로 유의하지 않음
* 통상 Residual deviance/ 자유도 < 2 이면 모형의 적합도에 큰 문제가 없다고 판단함

# 모형의 선택
* log(ㅠ/1-ㅠ) = B0 +B1X1 적합

In [ ]:
logit_m2 = glm(occurr ~ p_size_km, family = binomial(link = logit), data = glider)
summary(logit_m2)

In [ ]:
#X2를 추가하는 것이 적절한지 검정
anova(logit_m2, glider.logit_m1, test = 'Chisq')

* 0.305로 유의수준 0.05보다 크므로 유의하지 않음

In [ ]:
#이탈도에 근거해 모형적합도 평가
55.719/48

* 1에 가까워 모형적합도가 있다고 판단됨

In [ ]:
#AIC 비교
AIC(logit_m2, glider.logit_m1)

* m2(B0+B1X1) 모형이 더 적합함(m1 : B0+B1x1+B2x2)

In [ ]:
# stepAIC() : AIC값과 변수선택방향을 기준으로 최적모형 자동 선택
library(MASS)
stepAIC(glider.logit_m1, direction = 'both')

* 최종 로지스틱회귀모형 : ^B0 + ^B1X1 = -2.528 + 0.0022 X1

# 로지스틱함수의 형태

* pi(x)(성공률) = exp(B0 + B1x)/1+exp(B0 + B1x)
* 0 < pi(x) < 1 : 회귀계수 추정 시 제약조건없이 추정 가능

In [ ]:
#구획크기에 대한 출현여부 & 추정된 ^ pi(x)
p_size = seq(20, 230, 1)
hat_eta = predict(logit_m2, list(p_size_km = p_size), type = 'link')
par(mfrow = c(1, 2))
plot(glider$p_size_km, glider$occurr, xlab = 'p_size(x)', ylab = 'hat pi(x)\\ occurr', sub = '(a)', pch = 20)
lines(p_size, exp(hat_eta)/(1+exp(hat_eta)), lwd = 1.5, col = 'red')

#표본비율 & 추정된 ^pi(x)
glider_g_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/sugar_glider_binomial_g.csv'
glider_g = read.csv(glider_g_url)
plot(glider_g$p_size_med, glider_g$cases/glider_g$count, xlab = 'p_size(x)', ylim = c(0,1), ylab = 'hat pi(x) \\ sample prop', sub = '(b)', pch = 20, col = 'blue')
lines(p_size, exp(hat_eta)/(1+exp(hat_eta)), lwd = 1.5, col = 'red')

* (a) : 0 / 1값을 가지는 이항변수인 출현여부(occurr)를 로지스틱함수로 적합시키는 것이 자연스럽게 보이지 않음
* (b) : 구획크기를 적당한 계급구간으로 나누고 해당 계급구간에 있는 구획의 수, 출현한 구획의 수, 이들의 표본비율을 정리한 자료로 로지스틱함수 적합시킴


> glider_g
* 통상적인 이항자료는 해당 자료와 같이 설명변수의 각 수준에서 전체 관측도수와 관심사건의 발생 수로 정리되어 있다
* 이런 자료의 경우 로지스틱회귀모형을 적합시키려면 개별 개체에서 0/1ㅇ이 관측되는 경우와 다른 방식으로 반응변수를 설정해야됨
* 반응변수 => 설명변수의 각 수준에서 두가지값(관심사건수, 전체사건수- 관심사건수)을 cbind()로 결합

In [ ]:
#glider_g 회귀적합
head(glider_g)

In [ ]:
y = cbind(glider_g$cases, glider_g$count - glider_g$cases)
logit_mg = glm(y ~ glider_g$p_size_med, family = binomial(link=logit))
summary(logit_mg)

* 적합된 로지스틱회귀모형 : -2.539 + 0.022 X1
* 원자료로 분석한 결과와 거의 같음

#로지스틱회귀 승산비(odds ratio), 성공확률
* odds = pi(x) / 1-pi(x)


In [ ]:
#구획의 크기가 1km가 증가할 때 승산의 증가비, 95%신뢰구간 추정
exp(coef(logit_m2))

In [ ]:
exp(confint(logit_m2, parm = 'p_size_km', level = 0.95))

* 1km 증가 시 승산은 약 1.022배 증가
* 95% 신뢰수준에서 승산은 1.009 ~ 1.038배 사이에서 증가할 것으로 추정됨

In [ ]:
#x값(구획크기)에 따라 출현확률 구하기
x = 150
predict(logit_m2, list(p_size_km = x), type = 'response')

In [ ]:
#pi(x)의 95% 신뢰구간
